In [1]:
import json
from PIL import Image, ImageDraw

def f01(x, y, v0, v1):
    return (v0[1] - v1[1]) * x + (v1[0] - v0[0]) * y + v0[0] * v1[1] - v1[0] * v0[1]

def f12(x, y, v1, v2):
    return (v1[1] - v2[1]) * x + (v2[0] - v1[0]) * y + v1[0] * v2[1] - v2[0] * v1[1]

def f20(x, y, v2, v0):
    return (v2[1] - v0[1]) * x + (v0[0] - v2[0]) * y + v2[0] * v0[1] - v0[0] * v2[1]

def inside_triangle(x, y, v0, v1, v2):
    denominator = f12(v2[0], v2[1], v0, v1)
    if denominator == 0:
        return False

    alpha = f12(x, y, v0, v1) / denominator
    beta = f20(x, y, v1, v2) / denominator
    gamma = f01(x, y, v2, v0) / denominator

    return 0 <= alpha <= 1 and 0 <= beta <= 1 and 0 <= gamma <= 1

def main():
    # Load teapot data from JSON
    with open('teapot.json') as json_file:
        teapot_data = json.load(json_file)['data']

    # Create an image to render the teapot
    width, height = 256, 256
    image = Image.new("RGB", (width, height), "white")
    draw = ImageDraw.Draw(image)

    for triangle in teapot_data:
        v0 = (triangle['v0']['v'][0], triangle['v0']['v'][1])
        v1 = (triangle['v1']['v'][0], triangle['v1']['v'][1])
        v2 = (triangle['v2']['v'][0], triangle['v2']['v'][1])

        # Clip vertices to the buffer
        v0 = (max(0, min(v0[0], width - 1)), max(0, min(v0[1], height - 1)))
        v1 = (max(0, min(v1[0], width - 1)), max(0, min(v1[1], height - 1)))
        v2 = (max(0, min(v2[0], width - 1)), max(0, min(v2[1], height - 1)))

        # Calculate bounding box
        xmin, xmax = min(v0[0], v1[0], v2[0]), max(v0[0], v1[0], v2[0])
        ymin, ymax = min(v0[1], v1[1], v2[1]), max(v0[1], v1[1], v2[1])

        # Scan convert and fill the triangle
        for y in range(int(ymin), int(ymax) + 1):
            for x in range(int(xmin), int(xmax) + 1):
                if inside_triangle(x, y, v0, v1, v2):
                    draw.point((x, y), fill="black")

    # Save or display the result
    image.save("teapot_black_output.png")
    image.show()

if __name__ == "__main__":
    main()


In [2]:
import json
import numpy as np
from PIL import Image, ImageDraw

def load_triangle_data(filename):
    with open(filename, 'r') as file:
        data = json.load(file)
    return data["data"]

def line_eq(v0, v1, x, y):
    return (v0[1] - v1[1]) * x + (v1[0] - v0[0]) * y + (v0[0] * v1[1] - v1[0] * v0[1])

def compute_triangle_color(normal):
    dotp = 0.707 * normal[0] + 0.5 * normal[1] + 0.5 * normal[2]
    dotp = abs(dotp) if dotp < 0 else (1.0 if dotp > 1.0 else dotp)
    tri_color = [int(255 * 0.95 * dotp), int(255 * 0.65 * dotp), int(255 * 0.88 * dotp)]
    return tuple(tri_color)

def clip_and_clamp_coordinates(v0, v1, v2, canvas_width, canvas_height):
    xmin, xmax = max(np.floor(min(v0[0], v1[0], v2[0])), 0), min(np.ceil(max(v0[0], v1[0], v2[0])), canvas_width - 1)
    ymin, ymax = max(np.floor(min(v0[1], v1[1], v2[1])), 0), min(np.ceil(max(v0[1], v1[1], v2[1])), canvas_height - 1)
    return int(xmin), int(xmax), int(ymin), int(ymax)

def initialize_image_and_z_buffer(canvas_size):
    image = Image.new("RGB", (canvas_size, canvas_size), "black")
    z_buffer = np.full((canvas_size, canvas_size), np.inf)
    return image, z_buffer

def render_triangle(triangle, draw, z_buffer):
    v0, v1, v2 = triangle["v0"]["v"], triangle["v1"]["v"], triangle["v2"]["v"]
    normal = np.array(triangle["v0"]["n"])
    color = compute_triangle_color(normal)

    canvas_width, canvas_height = draw.im.size

    xmin, xmax, ymin, ymax = clip_and_clamp_coordinates(v0, v1, v2, canvas_width, canvas_height)

    f12_v0 = line_eq(v1, v2, v0[0], v0[1])
    f20_v1 = line_eq(v2, v0, v1[0], v1[1])
    f01_v2 = line_eq(v0, v1, v2[0], v2[1])

    for y in range(ymin, ymax + 1):
        for x in range(xmin, xmax + 1):
            alpha = line_eq(v1, v2, x, y) / f12_v0
            beta = line_eq(v2, v0, x, y) / f20_v1
            gamma = line_eq(v0, v1, x, y) / f01_v2

            if 0 <= alpha <= 1 and 0 <= beta <= 1 and 0 <= gamma <= 1:
                z = alpha * v0[2] + beta * v1[2] + gamma * v2[2]
                if z < z_buffer[y, x]:
                    draw.point((x, y), fill=color)
                    z_buffer[y, x] = z

def main():
    canvas_size = 256
    image, z_buffer = initialize_image_and_z_buffer(canvas_size)

    filename = 'teapot.json'
    triangles = load_triangle_data(filename)

    draw = ImageDraw.Draw(image)

    for triangle in triangles:
        render_triangle(triangle, draw, z_buffer)

    image.save("teapot_color_output.png")
    image.show()

if __name__ == "__main__":
    main()
